In [14]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout,Input,Flatten,Dense,MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [15]:
tf.test.is_gpu_available()

False

In [16]:
tf.config.list_physical_devices('GPU')

[]

In [17]:
batchsize=1

In [18]:
train_datagen= ImageDataGenerator(rescale=1./255, rotation_range=0.2,
                              zoom_range=0.2, width_shift_range=0.2,
                              height_shift_range=0.2, validation_split=0.2)

train_data=train_datagen.flow_from_directory(r'C:\Users\Admin\Desktop\project_new\Prepared_Data\Train'
                                            ,target_size=(80,80),batch_size=batchsize,class_mode='categorical', subset='training')

validation_data= train_datagen.flow_from_directory(r'C:\Users\Admin\Desktop\project_new\Prepared_Data\Train'
                                            ,target_size=(80,80),batch_size=batchsize,class_mode='categorical', subset='validation')


Found 64593 images belonging to 2 classes.
Found 16147 images belonging to 2 classes.


In [19]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_data = test_datagen.flow_from_directory(r'C:\Users\Admin\Desktop\project_new\Prepared_Data\Test'
                                            ,target_size=(80,80),batch_size=batchsize,class_mode='categorical')

Found 4158 images belonging to 2 classes.


In [20]:
bmodel = InceptionV3(include_top=False, weights='imagenet', input_tensor=Input(shape=(80,80,3), batch_size=batchsize))
hmodel = bmodel.output
hmodel = Flatten()(hmodel)
hmodel = Dense(64, activation='relu')(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(2,activation= 'softmax')(hmodel)

model = Model(inputs =bmodel.input, outputs= hmodel)
for layer in bmodel.layers:
    layer.trainable = False 

In [21]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(1, 80, 80, 3)]     0           []                               
                                                                                                  
 conv2d_94 (Conv2D)             (1, 39, 39, 32)      864         ['input_2[0][0]']                
                                                                                                  
 batch_normalization_94 (BatchN  (1, 39, 39, 32)     96          ['conv2d_94[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_94 (Activation)     (1, 39, 39, 32)      0           ['batch_normalization_94[0]

In [22]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping, ReduceLROnPlateau

In [23]:
checkpoint = ModelCheckpoint(r'C:\Users\Admin\Desktop\project_new\All Code Files\models',
                            monitor='val_loss',save_best_only=True,verbose=3)

earlystop = EarlyStopping(monitor = 'val_loss', patience=7, verbose= 3, restore_best_weights=True)

learning_rate = ReduceLROnPlateau(monitor= 'val_loss', patience=3, verbose=3)

callbacks = [checkpoint,earlystop,learning_rate]

In [24]:

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])


model.fit_generator(train_data,steps_per_epoch=train_data.samples//batchsize,
                   validation_data=validation_data,
                  validation_steps=validation_data.samples//batchsize,
                  callbacks=callbacks,
                   epochs=5)
#model.fit(train_data,epochs=5)


C:\Users\Admin\AppData\Local\Temp\ipykernel_9576\1429840191.py:4: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_data,steps_per_epoch=train_data.samples//batchsize,


Epoch 1/5
64593/64593 [==============================] - ETA: 0s - loss: 0.2362 - accuracy: 0.9108

KeyboardInterrupt: 

In [ ]:
#Model Evalution

In [ ]:
acc_tr, loss_tr = model.evaluate_generator(train_data)
print(acc_tr)
print(loss_tr)

In [ ]:
acc_vr, loss_vr = model.evaluate_generator(validation_data)
print(acc_vr)
print(loss_vr)

In [ ]:
acc_test, loss_test = model.evaluate_generator(test_data)
print(acc_test)
print(loss_test)